# **DESARROLLO DE MEDICAMENTOS EMPLEANDO INTELIGENCIA ARTIFICIAL**

 GRADO EN INGENIERÍA DE LA SALUD - *Trabajo Fin de Grado* 

Realizado por: *Beatriz García Dueñas* 

Tutor: *Juan Antonio Nepomuceno Chamorro*

Departamento: Lenguajes y Sistemas Informáticos


## **EXPERIMENTACIÓN CON DATASET**

El primer paso a la hora de empezar a aplicar las técnicas de Deep Learning es seleccionar el conjunto de datos con el que vamos a trabajar. Este paso es esencial ya que es con el que vamos a entrenar nuestro modelo.

Para seleccionar el conjunto de datos adecuado a nuestro problema primero hay que conocer el formato requerido y cómo se representa la información en dicho conjunto. Es necesario entender su estructura y complejidad ya que puede ser inteligible a simple vista. Como hemos visto anteriormente, en este caso vamos a trabajar con datos de tipo SMILES.

Todos los conjuntos de datos empleados en este trabajo han sido obtenidos a partir de la web MoleculeNet, una base de datos especialmente diseñada para proporcionar conjuntos moleculares y sus correspondientes propiedades a métodos de aprendizaje automático.

Para entender como trabaja la librería DeepChem con los conjuntos de datos experimentaremos con algunos ejemplos.

Para comenzar, cargamos el conjunto de datos de solubilidades escogido, el dataset Delaney.

In [ ]:
!pip install --pre deepchem
import deepchem as dc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 693 kB 3.9 MB/s 
     |████████████████████████████████| 29.5 MB 1.7 MB/s 


In [ ]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets


En primer lugar, observamos el parámetro que contiene la función de carga del conjunto de datos de Delaney. Las moléculas pueden representarse de muchas maneras y a través del parámetro “featurizer” podemos indicar cuál es la representación que queremos utilizar. 

La variable datasets que será nuestro conjunto de datos contiene tres objetos: el conjunto de entrenamiento, el conjunto de validación y el conjunto de prueba. Veamos cada uno en detalle.

In [ ]:
print(test_dataset)

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>


 “DiskDataset” es indica realmente un conjunto de datos que se ha guardado en el disco. Existen otros tipos de clases que almacenan datos como por ejemplo NumpyDataset, el cual almacena datos en memoria y sirve para manejar conjuntos de datos pequeños o medianos. Otros como ImageDataset almacena datos en archivos de imagen en el disco. 

En el interior del dataset nos encontramos una lista de muestras que en nuestro caso serían las moléculas. Para cada muestra, el conjunto de datos almacena la siguiente información:
-	Las características, denominadas “X”: corresponde a la entrada del modelo y representa a las muestras.
-	Las etiquetas, denominadas “Y”: durante el entrenamiento del modelo se intenta que la salida se acerque los más posible a Y.
-	Los pesos, denominados “w”: determina la importancia de cada entrada para el modelo. 
-	Un ID: es un identificador único para cada muestra. Para este conjunto de datos se corresponde con la cadena SMILES que describe la molécula.
-	Tasks_names: indica la información que contiene cada muestra. En nuestro caso las entradas son la log(solubilidad) en moles/litro. Hay conjuntos de datos que contienen diferente información para cada muestra y es en esta variable donde se indica.


Para acceder a estos datos individualmente, basta con indicar las propiedades: X, y, w e ids respectivamente. Hay que tener en cuenta que de esta forma estaremos accediendo al conjunto completo de datos, cosa que no sirve cuando no se han cargado todas las muestras del conjunto a la vez.

In [ ]:
test_dataset.y

array([[-1.60114461],
       [ 0.20848251],
       [-0.01602738],
       [-2.82191713],
       [-0.52891635],
       [ 1.10168349],
       [-0.88987406],
       [-0.52649706],
       [-0.76358725],
       [-0.64020358],
       [-0.38569452],
       [-0.62568785],
       [-0.39585553],
       [-2.05306753],
       [-0.29666474],
       [-0.73213651],
       [-1.27744393],
       [ 0.0081655 ],
       [ 0.97588054],
       [-0.10796031],
       [ 0.59847167],
       [-0.60149498],
       [-0.34988907],
       [ 0.34686576],
       [ 0.62750312],
       [ 0.14848418],
       [ 0.02268122],
       [-0.85310089],
       [-2.72079091],
       [ 0.42476682],
       [ 0.01300407],
       [-2.4851523 ],
       [-2.15516147],
       [ 1.00975056],
       [ 0.82588471],
       [-0.90390593],
       [-0.91067993],
       [-0.82455329],
       [ 1.26909819],
       [-1.14825397],
       [-2.1343556 ],
       [-1.15744727],
       [-0.1045733 ],
       [ 0.53073162],
       [-1.22567118],
       [-1

Una solución para grandes conjuntos de datos y que es muy empleada es la iteración sobre nuestro dataset: en cada iteración se cargará una parte de nuestros datos (lotes de muestras), se procesarán, y posteriormente se liberarán de la memoria antes de cargar la siguiente iteración. La función iterbatches permitirá realizar esta iteración sobre lotes de muestras. Además tendrá otras utilidades como iterar el conjunto completo varias veces (llamadas épocas) y cada vez con las muestras en un orden diferente.

In [ ]:
for X, y, w, ids in test_dataset.iterbatches(batch_size=50, epochs=10):
    print(y.shape)

(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)
(50, 1)
(50, 1)
(13, 1)


Otra forma de trabajar con los conjuntos de datos es crear data frames a partir de ellos. A través de la función todataframe() de Pandas es posible, sin embargo, requiere que los datos se almacenen en la memoria de una vez y, por tanto, solo sería recomendable para conjuntos de datos pequeños.

In [ ]:
test_dataset.to_dataframe()

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.821917,1.0,c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.528916,1.0,C1=Cc2cccc3cccc1c23
...,...,...,...,...
108,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.656304,1.0,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
109,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.743629,1.0,c1ccsc1
110,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.420799,1.0,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
111,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.209570,1.0,Cc1occc1C(=O)Nc2ccccc2
